In [1]:
import arcpy
arcpy.env.overwriteOutput = True 
from arcpy.sa import *
from arcpy import env

import os
import pandas as pd
import numpy as np


workspace = os.path.join("..", 'RESULTS')
#make sure workspace directory exists
if not os.path.exists(workspace):
    os.makedirs(workspace)
    
tempspace = os.path.join(workspace, "temp")
#make sure workspace directory exists
if not os.path.exists(tempspace):
    os.makedirs(tempspace)


outspace = os.path.join(workspace, 'Sens_test_Outputs')
#make sure workspace directory exists
if not os.path.exists(outspace):
    os.makedirs(outspace)

    
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container { width: 95%;} div#menubar-container { width: 85%; } div#maintoolbar-container { width: 99%; } </style> """))

In [2]:
#  path to the infrastructure layers 
InfraShpPath = os.path.join("..", 'Data/Raw/GIS/Infrastructure/Clipped')

Polyline_infrstructures = ['Relevant_Sewer_Lines_core.shp', 'ASPA_water_Distribution_core.shp', 
                           'ASPA_water_Transmission_core.shp', 'Roads_2006_core.shp'] 

Point_infrstructures = ['ASPA_CustomerMeters_core.shp', 'ASPA_water_Boosters_core.shp', 'All_sewer_LiftStations_core.shp', 'bldgs_pts_core.shp'] 

In [3]:
# Create shapefiles and a dic of the total lengths of polyline files 

OG_lenth_dic = {}
for i in Polyline_infrstructures:
    
    shapefile_to_measure = os.path.join(InfraShpPath, i)
    
    # First copy the original features to calculate and print the total lengths of polylines later
    arcpy.management.CopyFeatures(shapefile_to_measure, os.path.join(tempspace, "Length_{}".format(i)))

    fc_OG = os.path.join(tempspace, "Length_{}".format(i))
    arcpy.AddField_management(fc_OG, 'lenOG_m','DOUBLE')
    arcpy.management.CalculateField(fc_OG, "lenOG_m", "!SHAPE.geodesicLength@METERS!")   
    field = arcpy.da.TableToNumPyArray (fc_OG, "lenOG_m", skip_nulls=True)
    tot_len = field["lenOG_m"].sum()
    print("Total Original Length of {} is {} m".format(i, tot_len)) 
    OG_lenth_dic[i] = tot_len
    
    
for i in Point_infrstructures:
    fc = os.path.join(InfraShpPath, i)
    field = arcpy.da.TableToNumPyArray (fc, "FID", skip_nulls=True)
    tot_things = len(field)
    OG_lenth_dic[i] = tot_things
    print("Total number of {} is {}".format(i, tot_things))
    

# Turn the Dic into a dataframe
OG_lenth = pd.DataFrame.from_dict([OG_lenth_dic], orient='columns').transpose()
OG_lenth = OG_lenth.rename(columns={0:"Original_length_or_number"})
OG_lenth

Total Original Length of Relevant_Sewer_Lines_core.shp is 42315.5045899074 m
Total Original Length of ASPA_water_Distribution_core.shp is 44057.93550498824 m
Total Original Length of ASPA_water_Transmission_core.shp is 47137.500621334475 m
Total Original Length of Roads_2006_core.shp is 76765.56681189116 m
Total number of ASPA_CustomerMeters_core.shp is 1329
Total number of ASPA_water_Boosters_core.shp is 5
Total number of All_sewer_LiftStations_core.shp is 8
Total number of bldgs_pts_core.shp is 2398


,Original_length_or_number
ASPA_CustomerMeters_core.shp,1329.000000
ASPA_water_Boosters_core.shp,5.000000
ASPA_water_Distribution_core.shp,44057.935505
ASPA_water_Transmission_core.shp,47137.500621
All_sewer_LiftStations_core.shp,8.000000
Relevant_Sewer_Lines_core.shp,42315.504590
Roads_2006_core.shp,76765.566812
bldgs_pts_core.shp,2398.000000


In [4]:
SensTest_scenarios_dic = {}

for SenstestFile in os.listdir(os.path.join("..", "RESULTS", "Sens_test_outputs")):
    print(SenstestFile)
    scen_name = SenstestFile[:-4]
    
    # Create polygons for Surface or subsurface "flooded" areas for each of the scenarios in SLR_scenarios:
    in_raster = os.path.join("..", "RESULTS", "Sens_test_outputs", SenstestFile)
    outraster = os.path.join(tempspace, 'Temp_raster.tif')
    DaNewraster = Raster(in_raster) < 1.01                     #### Using only subsurface infrastructure for sens test 
    DaNewraster.save(outraster)

    # Convert the raster to polygons
    outPolygons = os.path.join(tempspace, 'temp.shp')
    arcpy.RasterToPolygon_conversion(outraster, outPolygons, "NO_SIMPLIFY")

    ## Delete any areas where the raster (now gridcode attribute) is 0
    arcpy.MakeFeatureLayer_management(outPolygons, "Temp_Poly")
    Selection = arcpy.SelectLayerByAttribute_management("Temp_Poly", "NEW_SELECTION", "gridcode > 0")
    arcpy.CopyFeatures_management(Selection, os.path.join(tempspace, 'FA_{}.shp'.format(scen_name)))

    results_dic = {}
    for file in os.listdir(InfraShpPath):
        if file.endswith(".shp"):
            shapefile_to_clip = os.path.join(InfraShpPath, file) 

            # Now do the actual clipping of flooded areas
            out_clipped  = os.path.join(tempspace, "{}_Flooded_{}".format(scen_name, file))
            arcpy.Clip_analysis(shapefile_to_clip, os.path.join(tempspace, 'FA_{}.shp'.format(scen_name)), out_clipped) 


            # NOW work on the Polyline_infrstructures:                                 
            if file in Polyline_infrstructures:
                arcpy.AddField_management(out_clipped, 'lenFlod_m','DOUBLE')
                arcpy.management.CalculateField(out_clipped, "lenFlod_m", "!SHAPE.geodesicLength@METERS!")   
                field = arcpy.da.TableToNumPyArray (out_clipped, "lenFlod_m", skip_nulls=True)
                flod_len = field["lenFlod_m"].sum()
                print("Length of flooded {} is {} m in {} at 1m depth".format(file, flod_len, scen_name)) 

                pct_flooded = round((flod_len/OG_lenth_dic[file])*100, 1)
                print("Total length of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))

                results_dic[file] = flod_len


           # NOW work on the point_infrstructures: 
            if file in Point_infrstructures:
                field = arcpy.da.TableToNumPyArray (out_clipped, "FID", skip_nulls=True)
                flood_things = len(field)
                print("Number of flooded {} is {}".format(file, flood_things))

                pct_flooded = round((flood_things/OG_lenth_dic[file])*100, 1)
                print("Total number of {} is {}, Percent of flooded is {}%  \n".format(file, OG_lenth_dic[file], pct_flooded))

                results_dic[file] = flood_things
                
        SensTest_scenarios_dic[scen_name] = results_dic     

D2W_A_Aua_MT_1.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded ASPA_water_Boosters_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1627.9228571578751 m in D2W_A_Aua_MT_1 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.7%  

Length of flooded ASPA_water_Transmission_core.shp is 1853.3616472042809 m in D2W_A_Aua_MT_1 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.9%  

Number of flooded bldgs_pts_core.shp is 49
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.0%  

Length of flooded Relevant_Sewer_Lines_co

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded ASPA_water_Boosters_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1594.711163662555 m in D2W_A_Fagaalu_MT_2 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.6%  

Length of flooded ASPA_water_Transmission_core.shp is 1813.2615484901512 m in D2W_A_Fagaalu_MT_2 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.8%  

Number of flooded bldgs_pts_core.shp is 48
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.0%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1413.6646510949772 m in D2W_A_Fagaalu_MT_2 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded 

Length of flooded ASPA_water_Distribution_core.shp is 1627.9228571578751 m in D2W_A_Laulii_MT_4 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.7%  

Length of flooded ASPA_water_Transmission_core.shp is 1853.3616472042809 m in D2W_A_Laulii_MT_4 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.9%  

Number of flooded bldgs_pts_core.shp is 49
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.0%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1461.7971693460172 m in D2W_A_Laulii_MT_4 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 3.5%  

Length of flooded Roads_2006_core.shp is 2929.177708660333 m in D2W_A_Laulii_MT_4 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 3.8%  

D2W_A_Laulii_MT_p25.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
T

Length of flooded ASPA_water_Transmission_core.shp is 1874.151387462581 m in D2W_A_minor_MT_p25 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 4.0%  

Number of flooded bldgs_pts_core.shp is 53
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.2%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1461.7971693460172 m in D2W_A_minor_MT_p25 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 3.5%  

Length of flooded Roads_2006_core.shp is 3046.232804882143 m in D2W_A_minor_MT_p25 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 4.0%  

D2W_A_minor_MT_p5.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded 

Number of flooded bldgs_pts_core.shp is 59
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.5%  

Length of flooded Relevant_Sewer_Lines_core.shp is 2000.613390331047 m in D2W_A_Utulei_MT_p5 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 4.7%  

Length of flooded Roads_2006_core.shp is 4707.960446175979 m in D2W_A_Utulei_MT_p5 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 6.1%  

D2W_A_Vaipito_MT_1.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded ASPA_water_Boosters_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1627.922857

...

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded ASPA_water_Boosters_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1627.9228571578751 m in D2W_Pago_inner_W_MT_1 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.7%  

Length of flooded ASPA_water_Transmission_core.shp is 1853.3616472042809 m in D2W_Pago_inner_W_MT_1 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.9%  

Number of flooded bldgs_pts_core.shp is 49
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.0%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1461.7971693460172 m in D2W_Pago_inner_W_MT_1 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent o

Number of flooded ASPA_CustomerMeters_core.shp is 21
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 1.6%  

Number of flooded ASPA_water_Boosters_core.shp is 1
Total number of ASPA_water_Boosters_core.shp is 5, Percent of flooded is 20.0%  

Length of flooded ASPA_water_Distribution_core.shp is 1560.929452871654 m in D2W_Pago_outer_MT_2 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 3.5%  

Length of flooded ASPA_water_Transmission_core.shp is 1728.5251527238859 m in D2W_Pago_outer_MT_2 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 3.7%  

Number of flooded bldgs_pts_core.shp is 46
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 1.9%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1327.7583752186672 m in D2W_Pago_outer_MT_2 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flood

Length of flooded ASPA_water_Distribution_core.shp is 435.56197597235 m in D2W_pump_multi_4 at 1m depth
Total length of ASPA_water_Distribution_core.shp is 44057.93550498824, Percent of flooded is 1.0%  

Length of flooded ASPA_water_Transmission_core.shp is 780.014698883622 m in D2W_pump_multi_4 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 1.7%  

Number of flooded bldgs_pts_core.shp is 23
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 1.0%  

Length of flooded Relevant_Sewer_Lines_core.shp is 331.78930402890796 m in D2W_pump_multi_4 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 0.8%  

Length of flooded Roads_2006_core.shp is 992.203364212123 m in D2W_pump_multi_4 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 1.3%  

D2W_pump_multi_p25.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
Total number

Length of flooded ASPA_water_Transmission_core.shp is 335.22484521243 m in D2W_rech_multi_p25 at 1m depth
Total length of ASPA_water_Transmission_core.shp is 47137.500621334475, Percent of flooded is 0.7%  

Number of flooded bldgs_pts_core.shp is 13
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 0.5%  

Length of flooded Relevant_Sewer_Lines_core.shp is 79.08293494067502 m in D2W_rech_multi_p25 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 0.2%  

Length of flooded Roads_2006_core.shp is 401.86945101633 m in D2W_rech_multi_p25 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 0.5%  

D2W_rech_multi_p5.asc
Number of flooded All_sewer_LiftStations_core.shp is 0
Total number of All_sewer_LiftStations_core.shp is 8, Percent of flooded is 0.0%  

Number of flooded ASPA_CustomerMeters_core.shp is 6
Total number of ASPA_CustomerMeters_core.shp is 1329, Percent of flooded is 0.5

Number of flooded bldgs_pts_core.shp is 49
Total number of bldgs_pts_core.shp is 2398, Percent of flooded is 2.0%  

Length of flooded Relevant_Sewer_Lines_core.shp is 1461.7971693460172 m in D2W_Trachyte_MT_p5 at 1m depth
Total length of Relevant_Sewer_Lines_core.shp is 42315.5045899074, Percent of flooded is 3.5%  

Length of flooded Roads_2006_core.shp is 2929.177708660333 m in D2W_Trachyte_MT_p5 at 1m depth
Total length of Roads_2006_core.shp is 76765.56681189116, Percent of flooded is 3.8%  



### Postprocess A: take the results dictionaries and make them dataframes 

In [5]:
SensTest_scenarios_dic["Total_Lengths"] = OG_lenth_dic     # Add in the total lengths/numbers

output = pd.DataFrame.from_dict(SensTest_scenarios_dic, orient='columns')   # make it all a datframe
output = output.transpose()   # Transpose it

# Create a dataframe of percentage flooded
output_pct = output.copy()
for col in output: 
    tot_things = output[col].loc['Total_Lengths']
    output_pct[col] = output_pct[col]/tot_things
    
# Format percentages 
output_pct['sum']  = output_pct.sum(axis=1)/7
output_pct = output_pct.round(3)*100

# Sort by best to worst scenario 
output_pct = output_pct.sort_values("sum")

# Sort the in m df by best to worst scenario  
output['tot_buried_lines'] = output[['ASPA_water_Distribution_core.shp', 'ASPA_water_Transmission_core.shp', 'Relevant_Sewer_Lines_core.shp']].sum(axis=1)
output = output.sort_values('tot_buried_lines')

### Postprocess B: Further consilidate into desired sensitivity test table

In [12]:
# Make a consolidated end factor, here is the average floodedness of the water and sewer lines
output["lines"] = output[['ASPA_water_Distribution_core.shp', 'ASPA_water_Transmission_core.shp', 'Relevant_Sewer_Lines_core.shp']].mean(axis=1)
output2 = output.reset_index()
output2 = output2[['index', 'lines']]

output2['index'] = output2['index'].apply(lambda x: x[4:])  # Pull out the D2W_thing
output2['Modfactor'] = output2['index'].apply(lambda x: x.split("_")[-1])   # Isolate the modfactor from the name
# arrange by infrastructure type
output2 = output2.sort_values('index')


# Group the dataframe by the modfactors 
grouped_df = output2.groupby("Modfactor")

temp_dic = {}
for key, item in grouped_df:
    # operate on each mini dataframe with like modfactors 
    want_col = item.copy()
    want_col = want_col.rename(columns={'lines': key})

    # Snazzy, this will strip out the modifactor from the end leaving just the infrastructure type
    want_col['index'] = want_col['index'].apply(lambda x: x.replace("_{}".format(key), ""))
    want_col = want_col[['index', key]]   # pull out relevant columns 
    #want_col = want_col.set_index('index')   # set index for the concat

    temp_dic[key] = want_col

# lengths is annoying and in there
if 'Lengths' in temp_dic.keys():    
    del temp_dic['Lengths']
    
# Now wrap it all into one dataframe 
df = pd.DataFrame(columns=["index"]) # Create an empty dataframe, this will be your final dataframe
for key in temp_dic:
    df = df.merge(temp_dic[key], on="index", how='outer') # Add your sub_df one by one

#reorder Columns 
df = df[['index', 'p25','p5', '1', '2', '4',]]

# set up percent difference frame
df_pct_diff = df[['index']].copy()
df_pct_diff["25% multiplier"] = round((-df["1"] + df["p25"])/df["1"]*100, 1)
df_pct_diff["50% multiplier"] = round((-df["1"] + df["p5"])/df["1"]*100, 1)
df_pct_diff["100% multiplier"] = 100
df_pct_diff["200% multiplier"] = round((-df["1"] + df["2"])/df["1"]*100, 1)
df_pct_diff["400% multiplier"] = round((-df["1"] + df["4"])/df["1"]*100, 1)

#Save csvs
df_pct_diff.to_csv(os.path.join("..", 'RESULTS', 'output_pct_sens_test.csv'))
df.to_csv(os.path.join("..", 'RESULTS', 'output_sens_test.csv'))

df_pct_diff

,index,25% multiplier,50% multiplier,100% multiplier,200% multiplier,400% multiplier
0,A_Aua_MT,8.9,4.0,100,-2.3,-4.3
1,A_Fagaalu_MT,11.6,3.9,100,-2.5,-4.9
2,A_Laulii_MT,0.0,0.0,100,0.0,0.0
3,A_Utulei_MT,68.3,35.4,100,-7.6,-9.2
4,A_Vaipito_MT,124.5,61.6,100,-67.3,-73.0
5,A_Vatia_MT,0.1,0.0,100,-0.2,-0.4
6,A_minor_MT,1.7,0.4,100,-0.7,-1.1
7,Dikes_MT,2.2,-0.4,100,0.0,-0.0
8,Pago_inner_E_MT,-0.8,-0.1,100,-0.3,-1.1
9,Pago_inner_W_MT,11.1,4.6,100,5.3,6.0
